In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.8-bin-hadoop2.7.tgz
!pip install findspark
!pip install kafka-python
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar"

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar pyspark-shell'

import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random
import pyspark
import sys
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from uuid import uuid1
from time import sleep
from json import dumps
from kafka import KafkaProducer
import requests, time, json

In [ ]:
!curl -sSOL  https://archive.apache.org/dist/kafka/2.7.0/kafka_2.13-2.7.0.tgz
!tar -xzf kafka_2.13-2.7.0.tgz

!./kafka_2.13-2.7.0/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-2.7.0/config/zookeeper.properties
!./kafka_2.13-2.7.0/bin/kafka-server-start.sh -daemon ./kafka_2.13-2.7.0/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

!./kafka_2.13-2.7.0/bin/kafka-topics.sh --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic ChemEng
!./kafka_2.13-2.7.0/bin/kafka-topics.sh --describe --bootstrap-server localhost:9092 --topic ChemEng

# **Alert porfolio**

In [ ]:
kafka_topic_name = "ChemEng"
kafka_bootstrap_servers = 'localhost:9092'
kafka_bootstrap_servers = 'xxxxxxxxxxx.compute.amazonaws.com:xxxxxxxxx'

In [ ]:
ssc.stop()
sc.stop()
sc = pyspark.SparkContext()
ssc = StreamingContext(sc,1)

kvs = KafkaUtils.createStream(ssc, kafka_bootstrap_servers, 'spark-streaming-consumer', {kafka_topic_name:1}) 
kvs = KafkaUtils.createDirectStream(ssc, [kafka_topic_name], {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'largest'})
#------------------------------------------------------------------------------------------------------------------------------
buy = 66570.3
P = 0.00045
N = -0.00045
#------------------------------------------------------------------------------------------------------------------------------
input = kvs.map(lambda x: [int(x[1].replace('"','').split(":")[0]), float(x[1].replace('"','').split(":")[1])])
# input.pprint()

alert_portP = input.map(lambda kv: 100*(kv[1]-buy)/buy,2).filter(lambda v: v<N).map(lambda v: '!!alert!! now your portfolio '+format(v, ".2f")+' %')
alert_portP.pprint()

alert_portN = input.map(lambda kv: 100*(kv[1]-buy)/buy,2).filter(lambda v: v>P).map(lambda v: '!!alert!! now your portfolio '+format(v, ".2f")+' %')
alert_portN.pprint()

ssc.start()
ssc.awaitTerminationOrTimeout(30)
ssc.stop()
sc.stop()

-------------------------------------------
Time: 2021-11-10 12:50:36
-------------------------------------------

-------------------------------------------
Time: 2021-11-10 12:50:36
-------------------------------------------

-------------------------------------------
Time: 2021-11-10 12:50:38
-------------------------------------------

-------------------------------------------
Time: 2021-11-10 12:50:38
-------------------------------------------

-------------------------------------------
Time: 2021-11-10 12:50:39
-------------------------------------------

-------------------------------------------
Time: 2021-11-10 12:50:39
-------------------------------------------

-------------------------------------------
Time: 2021-11-10 12:50:40
-------------------------------------------

-------------------------------------------
Time: 2021-11-10 12:50:40
-------------------------------------------

-------------------------------------------
Time: 2021-11-10 12:50:41
----------

# **Price Alert**

In [ ]:
kafka_topic_name = "ChemEng"
kafka_bootstrap_servers = 'localhost:9092'
kafka_bootstrap_servers = 'ec2-13-229-46-113.ap-southeast-1.compute.amazonaws.com:9092'

In [ ]:
ssc.stop()
sc.stop()
sc = pyspark.SparkContext()
ssc = StreamingContext(sc,1)

kvs = KafkaUtils.createStream(ssc, kafka_bootstrap_servers, 'spark-streaming-consumer', {kafka_topic_name:1}) 
kvs = KafkaUtils.createDirectStream(ssc, [kafka_topic_name], {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'largest'})

producer = KafkaProducer(bootstrap_servers=[kafka_bootstrap_servers],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))
#------------------------------------------------------------------------------------------------------------------------------
input = kvs.map(lambda x: [int(x[1].replace('"','').split(":")[0]), float(x[1].replace('"','').split(":")[1])])
# input.pprint()

# show = input.map(lambda x: x[1])
# show.pprint()

sum_price = input.map(lambda kv: kv[1]).reduce(lambda a,b: a+b).map(lambda x: (0,x))
# sum_price.pprint()

count_price = input.count().map(lambda x: (0,x))
# count_price.pprint()

# avg_price = sum_price.fullOuterJoin(count_price).map(lambda x: x[1][0]/x[1][1]).map(lambda x: "average = "+str(x))
avg_price = sum_price.fullOuterJoin(count_price).map(lambda x: [0,x[1][0]/x[1][1]])
# avg_price.pprint()

adN = input.map(lambda x: [0,x[1]]).fullOuterJoin(avg_price).map(lambda x: 100*x[1][0]/x[1][1]-100*x[1][1]/x[1][1]).filter(lambda x:x<-30).map(lambda v: '!!alert!! price')
adN.pprint()
adP = input.map(lambda x: [0,x[1]]).fullOuterJoin(avg_price).map(lambda x: 100*x[1][0]/x[1][1]-100*x[1][1]/x[1][1]).filter(lambda x:x>30).map(lambda v: '!!alert!! price')
adP.pprint()

ssc.start()
ssc.awaitTerminationOrTimeout(20)
ssc.stop()
sc.stop()

-------------------------------------------
Time: 2021-11-10 13:31:07
-------------------------------------------

-------------------------------------------
Time: 2021-11-10 13:31:07
-------------------------------------------

-------------------------------------------
Time: 2021-11-10 13:31:08
-------------------------------------------

-------------------------------------------
Time: 2021-11-10 13:31:08
-------------------------------------------

-------------------------------------------
Time: 2021-11-10 13:31:09
-------------------------------------------

-------------------------------------------
Time: 2021-11-10 13:31:09
-------------------------------------------

-------------------------------------------
Time: 2021-11-10 13:31:10
-------------------------------------------

-------------------------------------------
Time: 2021-11-10 13:31:10
-------------------------------------------

-------------------------------------------
Time: 2021-11-10 13:31:11
----------